<table align="left">
<tr>

<th, style="background-color:white">
<img src="https://github.com/mlgill/ODSC_East_2017_PythonNLP/blob/master/assets/logo.png?raw=true", width=140, height=100>
</th>

<th, style="background-color:white">
<div align="left">
<h1>Learning from Text: <br> Introduction to Natural Language Processing with Python</h1>  
<h2>Michelle L. Gill, Ph.D.</h2>     
Senior Data Scientist, Metis  
ODSC East  
May 3, 2017 
</div>
</th>

</tr>
</table>  

## LDA Walkthrough and Exercises

## The Data

We will be using a portion data set containing approximately 20,000 posts partitioned evenly across 20 different newsgroups. This data set is quite famous. We will be using a sample of this data set, containing 5 topics and about 3,000 posts.

We will begin by loading the data.

In [1]:
import nltk
from accessory_functions import nltk_path

# Setup nltk corpora path
nltk.data.path.insert(0, nltk_path)

In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

topic_list = ['sci.space', 'comp.sys.mac.hardware', 'rec.autos',
              'rec.sport.baseball', 'sci.med']

dataset = fetch_20newsgroups(shuffle=True, random_state=1, data_home='../data',
                             categories=topic_list,
                             remove=('headers', 'footers', 'quotes'))

data = pd.DataFrame(dataset['data'], columns=['text'])
print(len(data))

2956


## Preprocess the Data

Next we will preprocess the data using the convenience method from `accessory_functions`.

In [3]:
from accessory_functions import preprocess_series_text

data['text'] = preprocess_series_text(data.text, 
                                      nltk_path=nltk_path)

In [4]:
data.head()

,text
0,otoh u get lucky unplugged replugged scsi adb ...
1,yes everyone else may wonder fred well would o...
2,umm perhaps could explain right talk
3,like alomar like differ opinion city likely po...
4,wow know uranus long way think far away


## Create Numerical Features

Use Count Vectorizer to create a document-term matrix.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

n_features = 1000
cv = CountVectorizer(max_df=0.95, min_df=2, 
                     max_features=n_features)
X = cv.fit_transform(data.text)

print(X.shape)

(2956, 1000)


In [6]:
pd.DataFrame(X.toarray(), columns=cv.get_feature_names()).head()

,ab,ability,able,ac,accept,access,accord,acid,act,activity,...,write,wrong,yankee,yeah,year,yeast,yes,yet,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create an LDA Model

Use Scikit-learn's [`LatentDirichletAllocation`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) to fit an LDA model.

In [7]:
LatentDirichletAllocation?

Object `LatentDirichletAllocation` not found.


In [8]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = len(topic_list)
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

## Print Top Words
Print the top words associated with each topic.

In [9]:
def print_top_words(model, feature_names, n_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % (topic_idx+1))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [10]:
n_top_words = 20

print("Topics in LDA model:")
cv_feature_names = cv.get_feature_names()
print_top_words(lda, cv_feature_names, n_top_words)

Topics in LDA model:
Topic #1:
one would like get people think know make thing cause say see use food many could well time take much
Topic #2:
year game get good go last think would well team run one hit player time first like win play say
Topic #3:
patient use health post edu medical information group number disease software mail report new pain keyboard cancer center program please
Topic #4:
car get use would drive one mac like problem know go work new apple time good thanks make think system
Topic #5:
space launch nasa edu satellite system orbit data com mission earth use program shuttle lunar year also moon rocket first



## Visualize the LDA Model

A visualization of the topic model can be easily created with `pyLDAvis`. 

In [11]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, X, cv)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      28.069957        1       1 -0.054177  0.025913
0      21.783139        1       2 -0.077119  0.041440
4      20.084867        1       3  0.160342 -0.116085
1      16.739182        1       4 -0.161389 -0.057071
2      13.322855        1       5  0.132344  0.105803, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
839   Default   980.000000        space   980.000000  30.0000  30.0000
129   Default   706.000000          car   706.000000  29.0000  29.0000
459   Default   433.000000       launch   433.000000  28.0000  28.0000
352   Default   362.000000         game   362.000000  27.0000  27.0000
760   Default   354.000000    satellite   354.000000  26.0000  26.0000
564   Default   365.000000         nasa   365.000000  25.0000  25.0000
994   Default  1023.000000         year  1023.000000  24.0000  24.0000
494   Default   413.000000          mac   413.000000  23.0000  23.0000
624   Default   233.000000      patient   233.000000  22.0000  22.0000
259   Default   651.000000          edu   651.000000  21.0000  21.0000
891   Default   305.000000         team   305.000000  20.0000  20.0000
246   Default   490.000000        drive   490.000000  19.0000  19.0000
606   Default   259.000000        orbit   259.000000  18.0000  18.0000
389   Default   230.000000          hit   230.000000  17.0000  17.0000
379   Default   210.000000       health   210.000000  16.0000  16.0000
651   Default   212.000000       player   212.000000  15.0000  15.0000
522   Default   208.000000      medical   208.000000  14.0000  14.0000
544   Default   222.000000      mission   222.000000  13.0000  13.0000
754   Default   431.000000          run   431.000000  12.0000  12.0000
979   Default   189.000000          win   189.000000  11.0000  11.0000
906   Default   892.000000        think   892.000000  10.0000  10.0000
456   Default   454.000000         last   454.000000   9.0000   9.0000
681   Default   363.000000      program   363.000000   8.0000   8.0000
800   Default   202.000000      shuttle   202.000000   7.0000   7.0000
650   Default   196.000000         play   196.000000   6.0000   6.0000
332   Default   230.000000         food   230.000000   5.0000   5.0000
493   Default   188.000000        lunar   188.000000   4.0000   4.0000
416   Default   321.000000  information   321.000000   3.0000   3.0000
254   Default   241.000000        earth   241.000000   2.0000   2.0000
643   Default   163.000000        pitch   163.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
431    Topic5    58.825462     internet    88.794877   1.6039  -5.7539
417    Topic5    55.512113       injury    85.944404   1.5786  -5.8119
864    Topic5    42.741629       street    66.466306   1.5742  -6.0733
369    Topic5   143.365636        group   227.372237   1.5545  -4.8631
245    Topic5    82.455948           dr   133.224033   1.5359  -5.4162
823    Topic5   135.947613     software   235.212999   1.4675  -4.9162
929    Topic5    82.625906    treatment   134.978976   1.5249  -5.4142
416    Topic5   163.841610  information   321.722327   1.3409  -4.7296
732    Topic5   135.500195       report   264.007112   1.3487  -4.9195
659    Topic5   191.151582         post   435.948509   1.1912  -4.5754
66     Topic5   110.297358      article   202.651432   1.4074  -5.1253
498    Topic5   135.606310         mail   283.970337   1.2766  -4.9187
239    Topic5   110.628952       doctor   208.138550   1.3837  -5.1223
138    Topic5   124.984914       center   273.800096   1.2315  -5.0003
15     Topic5    95.556652      address   179.814041   1.3835  -5.2688
259    Topic5   182.526150          edu   651.899018   0.7427  -4.6216
589    Topic5   137.195754       number   369.129991   1.0259  -4.9071
935    Topic5   106.468661   

## Question

* Fit an LDA model with a different number of topics and compare the top 20 words to those from the model above.
* Create a different document-term matrix by changing input parameters (max_features, etc.) or by switching to `TfidfVectorizer` and use this to fit another LDA model